- %env PYTORCH_ENABLE_MPS_FALLBACK=1 # MPS fallback for GPUs that don't support it
- %env export CUDA_VISIBLE_DEVICES="" # completely disable GPU and run on CPU only

In [1]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1
%env CUDA_VISIBLE_DEVICES=""

env: PYTORCH_ENABLE_MPS_FALLBACK=1
env: CUDA_VISIBLE_DEVICES=""


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import lightning as L
import torch


from SoccerNet.Evaluation.MV_FoulRecognition import evaluate
from torch.utils.data import DataLoader, random_split

from src.training import TrainingConfig
from src.dataset import MultiViewDataset
from src.model import LitMVNNetwork, get_pre_model
from src.loss import get_criterion
from src.eval import save_evaluation_file
from datetime import datetime
import os
from src.augment import get_augmentation
import wandb
from lightning.pytorch.loggers import WandbLogger

In [4]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: milosz-l (wut-zzsn). Use `wandb login --relogin` to force relogin


True

In [5]:
torch.cuda.empty_cache()
torch.mps.empty_cache()

In [6]:
num_epochs = 10
start_frame = 0
end_frame = 64
fps = 25
num_views = 5
pre_model = "s3d"
max_num_worker = 0
batch_size = 2
data_aug = False
pooling_type = "max"
weight_decay = 0.001
step_size = 3
gamma = 0.1
LR = 0.01
weighted_loss = False
data_aug = False

training_config = TrainingConfig(
    start_frame=start_frame,
    end_frame=end_frame,
    fps=fps,
    num_views=num_views,
    pre_model=pre_model,
    max_num_worker=max_num_worker,
    batch_size=batch_size,
    data_aug=data_aug,
    pooling_type=pooling_type,
    weight_decay=weight_decay,
    step_size=step_size,
    gamma=gamma,
    LR=LR,
    weighted_loss=weighted_loss,
)

In [7]:
wandb.init(
    project="ZZSN multi-view-foul-recognition", config=training_config.model_dump()
)

In [8]:
path = "data/mvfouls"
predictions_output_dir = "outputs"

transform_aug = get_augmentation(training_config.data_aug)
transforms_model = get_pre_model(training_config.pre_model)

In [9]:
dataset_Train = MultiViewDataset(
    path=path,
    start=start_frame,
    end=end_frame,
    fps=fps,
    split="Train",
    num_views=5,
    transform=transform_aug,
    transform_model=transforms_model,
)

train_size = int(0.7 * len(dataset_Train))
val_size = len(dataset_Train) - train_size

train_set, val_set = random_split(dataset_Train, [train_size, val_size])
train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=max_num_worker,
    pin_memory=True,
)
val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=max_num_worker,
    pin_memory=True,
)

2319


In [10]:
dataset_dir = "datasets"

dataset_Test = MultiViewDataset(
    path=path,
    start=start_frame,
    end=end_frame,
    fps=fps,
    split="Test",
    num_views=5,
    transform_model=transforms_model,
)


# dataset_Chall = MultiViewDataset(
#     path=path,
#     start=start_frame,
#     end=end_frame,
#     fps=fps,
#     split="Chall",
#     num_views=5,
#     transform_model=transforms_model,
# )

test_loader = torch.utils.data.DataLoader(
    dataset_Test,
    batch_size=1,
    shuffle=False,
    num_workers=max_num_worker,
    pin_memory=True,
)

# chall_loader = torch.utils.data.DataLoader(
#     dataset_Chall,
#     batch_size=1,
#     shuffle=False,
#     num_workers=max_num_worker,
#     pin_memory=True,
# )

251


In [11]:
criterion = get_criterion(weighted_loss, dataset_train=dataset_Train)
model = LitMVNNetwork(
    pre_model=pre_model,
    pooling_type=pooling_type,
    criterion=criterion,
    config=training_config,
)
job_id = str(datetime.now())
wand_logger = WandbLogger(log_model="all")

In [12]:
trainer = L.Trainer(limit_train_batches=3, max_epochs=num_epochs, logger=wand_logger)
trainer.fit(model=model, train_dataloaders=train_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/milosz/opt/miniconda3/envs/zzsn/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name  | Type      | Params
------------------------------------
0 | model | MVNetwork | 9.0 M 
------------------------------------
9.0 M     Trainable params
0         Non-trainable params
9.0 M     Total params
35.875    Total estimated model params size (MB)
/Users/milosz/opt/miniconda3/envs/zzsn/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` 

Epoch 0:   0%|          | 0/3 [00:00<?, ?it/s] 

/Users/milosz/opt/miniconda3/envs/zzsn/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
/Users/milosz/opt/miniconda3/envs/zzsn/lib/python3.10/site-packages/torch/nn/functional.py:882: UserWarning: The operator 'aten::max_pool3d_with_indices' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch.max_pool3d(input, kernel_size, stride, padding, dilation, ceil_mode)
/Users/milosz/opt/miniconda3/envs/zzsn/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_si

Epoch 3:  33%|███▎      | 1/3 [00:54<01:49,  0.02it/s, v_num=7z51, train_step_loss=46.20, train_epoch_loss=46.10]

/Users/milosz/opt/miniconda3/envs/zzsn/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [13]:
os.makedirs(predictions_output_dir, exist_ok=True)

test_set = f"test_{job_id}"
chall_set = f"chall_{job_id}"

test_prediction_file = save_evaluation_file(
    test_loader, model=model, set_name=test_set, output_dir=predictions_output_dir
)
chall_prediction_file = save_evaluation_file(
    chall_loader, model=model, set_name=chall_set, output_dir=predictions_output_dir
)

  0%|          | 0/251 [00:13<?, ?it/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
test_results = evaluate(
    os.path.join(path, "Test", "annotations.json"), test_prediction_file
)
wandb.log(test_results)

<class 'numpy.ndarray'>


In [ ]:
wandb.finish()

accuracy_action,▁
accuracy_offence_severity,▁
balanced_accuracy_action,▁
balanced_accuracy_offence_severity,▁
epoch,▁▁▅▅██
leaderboard_value,▁
train_epoch_loss,▁█▄
trainer/global_step,▁▁▂▃▃▃▃▃▅▅▄▅▅██
val_epoch_loss,▁
val_step_loss,▁▂█
accuracy_action,17.13147
